In [69]:
from nilearn import datasets, plotting
from nilearn.maskers import NiftiMapsMasker
import numpy as np
from tqdm import tqdm

In [53]:
development_dataset = datasets.fetch_development_fmri()

In [68]:
aal_data = datasets.fetch_atlas_msdl()

masker = NiftiMapsMasker(
    aal_data.maps,
    resampling_target="data",
    t_r=2,
    detrend=True,
    low_pass=0.1,
    high_pass=0.01,
    memory="nilearn_cache",
    memory_level=1,
    standardize="zscore_sample",
    standardize_confounds="zscore_sample",
).fit()


In [70]:
pooled_subjects = []
groups = []  # child or adult
for func_file, confound_file, phenotypic in tqdm(zip(
    development_dataset.func,
    development_dataset.confounds,
    development_dataset.phenotypic,
)):
    time_series = masker.transform(func_file, confounds=confound_file)
    pooled_subjects.append(time_series)
    groups.append(phenotypic["Child_Adult"])

155it [05:16,  2.04s/it]


In [73]:
X = np.stack(pooled_subjects)
np.save("Data/X", X)

In [85]:
labels = np.array([0 if g == 'adult' else 1 for g in groups])
np.save("Data/y", labels)